# Price

In [4]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

class PriceSimulation:
    """
    Create statistical price profiles with percentile compression for outliers
    Handles extreme prices elegantly while preserving their presence in distributions
    Compresses values outside P10-P90 range for cleaner statistics
    """
    
    def __init__(self):
        print("\n🔧 Initializing Price Simulation...")
        
        # Define paths - reading from resurety_data folder (UPDATED PATH)
        self.data_path = Path('resurety_data')  # Changed from '../resurety_data'
        self.base_output_path = Path('Renewable Portfolio LLC')  # New base output path
        
        # Get available combined files and create mapping
        self.available_files = list(self.data_path.glob('*_generation_price_combined.csv'))
        self.available_sites = []
        self.site_file_map = {}  # Map clean site names to actual filenames
        
        for f in self.available_files:
            # Store the full filename (without path)
            full_filename = f.name
            
            # Extract site name by removing '_generation_price_combined.csv'
            site_name = f.stem.replace('_generation_price_combined', '')
            
            # Clean up the site name by removing '_hourly' to avoid redundancy in output files
            clean_site_name = site_name.replace('_hourly', '')
            
            self.available_sites.append(clean_site_name)
            # Map the clean name to the actual filename
            self.site_file_map[clean_site_name] = full_filename
        
        print(f"   Found {len(self.available_files)} combined files")
        
        # Define percentiles for FULL distribution (including compressed outliers)
        self.full_percentiles = [1, 5, 10, 25, 50, 75, 90, 95, 99]
        
        # Define compression thresholds (P10 and P90 for compression)
        self.compression_lower = 10
        self.compression_upper = 90
        
        # Define thresholds for extreme event tracking
        self.negative_threshold = 0
        self.spike_threshold = 100
        self.extreme_spike_threshold = 200
        
        # Month names for labeling
        self.month_names = ['', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                           'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
        self.month_names_full = ['', 'January', 'February', 'March', 'April', 'May', 'June',
                                'July', 'August', 'September', 'October', 'November', 'December']
    
    def get_site_selection(self):
        """
        Interactive site selection with option for all sites
        """
        print("\n" + "="*60)
        print("PRICE SIMULATION WITH OUTLIER COMPRESSION")
        print("="*60)
        
        if not self.available_sites:
            print("❌ No combined generation-price files found in resurety_data!")
            return None
        
        print("\nAvailable options:")
        print("0. ALL SITES (Process all sites at once)")
        for i, site in enumerate(self.available_sites):
            print(f"{i+1}. {site}")
        
        print("="*60)
        
        while True:
            try:
                selection = input("\n💰 Select option number (0 for all sites): ").strip()
                if selection == '0':
                    return 'ALL_SITES'
                else:
                    idx = int(selection) - 1
                    if 0 <= idx < len(self.available_sites):
                        return self.available_sites[idx]
                    else:
                        print("❌ Invalid selection!")
            except:
                print("❌ Please enter a valid number!")
    
    def get_automatic_month_range(self):
        """
        Automatically determine month range: current month to 11 months later
        """
        current_date = datetime.now()
        current_month = current_date.month
        
        # Use current month as start
        start_month = current_month
        
        # End month is one month before start month (12 month cycle)
        if start_month == 1:
            end_month = 12
        else:
            end_month = start_month - 1
        
        print(f"\n📅 Auto-detected period: {self.month_names[start_month]} to {self.month_names[end_month]} (12 months)")
        print(f"   Starting from current month: {self.month_names[current_month]}")
        
        return start_month, end_month
    
    def get_months_in_range(self, start_month, end_month):
        """
        Get list of months in range, handling year-wrapping
        """
        if start_month <= end_month:
            return list(range(start_month, end_month + 1))
        else:
            return list(range(start_month, 13)) + list(range(1, end_month + 1))
    
    def filter_data_for_months(self, df, start_month, end_month):
        """
        Filter dataframe for month range, handling year-wrapping
        """
        months_in_range = self.get_months_in_range(start_month, end_month)
        return df[df['month'].isin(months_in_range)].copy()
    
    def create_month_order_map(self, start_month, end_month):
        """
        Create a mapping for sorting months in the specified order
        """
        months_in_range = self.get_months_in_range(start_month, end_month)
        return {month: idx for idx, month in enumerate(months_in_range)}
    
    def get_forecast_year(self, month):
        """
        Determine the forecast year for a given month based on current date
        """
        current_date = datetime.now()
        current_year = current_date.year
        current_month = current_date.month
        
        # If the forecast month is >= current month, it's this year
        # Otherwise it's next year (handling year wrap)
        if month >= current_month:
            return current_year
        else:
            return current_year + 1
    
    def compress_outliers(self, values, lower_pct=10, upper_pct=90):
        """
        Compress outliers using logarithmic compression
        This keeps outliers in the distribution but makes them manageable for visualization
        """
        # Calculate compression thresholds
        P_lower = np.percentile(values, lower_pct)
        P_upper = np.percentile(values, upper_pct)
        
        # Create compressed values array
        compressed = np.copy(values)
        
        # Compress lower tail (negative extremes)
        lower_mask = values < P_lower
        if np.any(lower_mask):
            # Use logarithmic compression for negative tail
            # Shift to make positive for log, then shift back
            offset = P_lower - values[lower_mask]
            compressed[lower_mask] = P_lower - np.log1p(offset)
        
        # Compress upper tail (positive extremes)
        upper_mask = values > P_upper
        if np.any(upper_mask):
            # Use logarithmic compression for positive tail
            offset = values[upper_mask] - P_upper
            compressed[upper_mask] = P_upper + np.log1p(offset)
        
        # Return compressed values and compression info
        compression_info = {
            'P_lower': P_lower,
            'P_upper': P_upper,
            'n_compressed_lower': np.sum(lower_mask),
            'n_compressed_upper': np.sum(upper_mask),
            'original_min': np.min(values),
            'original_max': np.max(values),
            'compressed_min': np.min(compressed),
            'compressed_max': np.max(compressed)
        }
        
        return compressed, compression_info
    
    def calculate_extreme_statistics(self, prices, generation=None):
        """
        Enhanced extreme statistics calculation
        """
        extreme_stats = {}
        
        # Overall distribution info
        extreme_stats['min'] = np.min(prices)
        extreme_stats['max'] = np.max(prices)
        extreme_stats['mean'] = np.mean(prices)
        extreme_stats['std'] = np.std(prices)
        
        # Negative price analysis
        negative_mask = prices < self.negative_threshold
        extreme_stats['negative_count'] = negative_mask.sum()
        extreme_stats['negative_pct'] = negative_mask.sum() / len(prices) * 100
        
        if negative_mask.sum() > 0:
            neg_prices = prices[negative_mask]
            extreme_stats['negative_mean'] = np.mean(neg_prices)
            extreme_stats['negative_worst'] = np.min(neg_prices)
            if generation is not None:
                extreme_stats['negative_avg_gen'] = np.mean(generation[negative_mask])
        
        # Spike analysis
        spike_mask = prices > self.spike_threshold
        extreme_stats['spike_count'] = spike_mask.sum()
        extreme_stats['spike_pct'] = spike_mask.sum() / len(prices) * 100
        
        if spike_mask.sum() > 0:
            spike_prices = prices[spike_mask]
            extreme_stats['spike_mean'] = np.mean(spike_prices)
            extreme_stats['spike_worst'] = np.max(spike_prices)
            if generation is not None:
                extreme_stats['spike_avg_gen'] = np.mean(generation[spike_mask])
        
        # Extreme spike count
        extreme_spike_mask = prices > self.extreme_spike_threshold
        extreme_stats['extreme_spike_count'] = extreme_spike_mask.sum()
        
        return extreme_stats
    
    def calculate_hourly_statistics(self, df_filtered, month_order_map):
        """
        Calculate hourly price statistics with percentile compression
        """
        print("\n⚡ Calculating HOURLY price statistics...")
        
        grouped = df_filtered.groupby(['month', 'day', 'hour'])
        
        results = []
        
        for (month, day, hour), group in grouped:
            if len(group) < 5:  # Skip if too few data points
                continue
            
            prices = group['price'].values
            generation = group['generation_mw'].values
            
            # Determine the forecast year
            forecast_year = self.get_forecast_year(month)
            
            # Apply percentile compression
            compressed_prices, compression_info = self.compress_outliers(
                prices, 
                self.compression_lower, 
                self.compression_upper
            )
            
            # Create datetime label
            datetime_label = f"{self.month_names[month]}-{day:02d} {hour:02d}:00"
            
            # Main statistics on COMPRESSED data for visualization
            stats = {
                'datetime_label': datetime_label,
                'year': forecast_year,  # ADD YEAR
                'month': month,
                'day': day,
                'hour': hour,
                'month_order': month_order_map[month],
                'mean': np.mean(compressed_prices),
                'std_dev': np.std(compressed_prices),
                'count': len(group),
                'avg_generation': np.mean(generation)
            }
            
            # Calculate percentiles on COMPRESSED data
            for p in self.full_percentiles:
                stats[f'p{p}'] = np.percentile(compressed_prices, p)
            
            # Also store ACTUAL percentiles for reference
            stats['actual_mean'] = np.mean(prices)
            stats['actual_min'] = np.min(prices)
            stats['actual_max'] = np.max(prices)
            stats['actual_p5'] = np.percentile(prices, 5)
            stats['actual_p10'] = np.percentile(prices, 10)
            stats['actual_p90'] = np.percentile(prices, 90)
            stats['actual_p95'] = np.percentile(prices, 95)
            
            results.append(stats)
        
        # Convert to DataFrames and sort
        results_df = pd.DataFrame(results)
        results_df = results_df.sort_values(['month_order', 'day', 'hour']).reset_index(drop=True)
        results_df = results_df.drop('month_order', axis=1)
        
        print(f"   ✓ Calculated statistics for {len(results_df)} hourly slots")
        
        return results_df
    
    def calculate_daily_statistics(self, df_filtered, month_order_map):
        """
        Calculate daily GENERATION-WEIGHTED price statistics with compression
        """
        print("\n📅 Calculating DAILY generation-weighted price statistics...")
        
        # First calculate generation-weighted prices for each day
        def calc_weighted_daily(group):
            mask = group['generation_mw'] > 0
            if mask.sum() == 0:
                return pd.Series({
                    'weighted_price': np.nan,
                    'total_generation': 0,
                    'generating_hours': 0
                })
            
            gen_positive = group.loc[mask]
            weighted_price = (gen_positive['price'] * gen_positive['generation_mw']).sum() / gen_positive['generation_mw'].sum()
            
            return pd.Series({
                'weighted_price': weighted_price,
                'total_generation': group['generation_mw'].sum(),
                'generating_hours': mask.sum()
            })
        
        # Calculate daily aggregates for each year
        daily_data = df_filtered.groupby(['year', 'month', 'day']).apply(calc_weighted_daily).reset_index()
        
        # Now calculate statistics across years for each day
        grouped = daily_data.groupby(['month', 'day'])
        
        results = []
        
        for (month, day), group in grouped:
            valid_group = group[group['weighted_price'].notna()]
            if len(valid_group) < 5:
                continue
            
            weighted_prices = valid_group['weighted_price'].values
            
            # Determine the forecast year
            forecast_year = self.get_forecast_year(month)
            
            # Apply compression
            compressed_prices, compression_info = self.compress_outliers(
                weighted_prices,
                self.compression_lower,
                self.compression_upper
            )
            
            # Create date label
            date_label = f"{self.month_names[month]}-{day:02d}"
            
            # Main statistics on compressed data
            stats = {
                'date_label': date_label,
                'year': forecast_year,  # ADD YEAR
                'month': month,
                'day': day,
                'month_order': month_order_map[month],
                'mean': np.mean(compressed_prices),
                'std_dev': np.std(compressed_prices),
                'count': len(valid_group),
                'avg_generation_mwh': valid_group['total_generation'].mean(),
                'avg_generating_hours': valid_group['generating_hours'].mean()
            }
            
            # Percentiles on compressed data
            for p in self.full_percentiles:
                stats[f'p{p}'] = np.percentile(compressed_prices, p)
            
            # Actual percentiles for reference
            stats['actual_mean'] = np.mean(weighted_prices)
            stats['actual_min'] = np.min(weighted_prices)
            stats['actual_max'] = np.max(weighted_prices)
            stats['actual_p5'] = np.percentile(weighted_prices, 5)
            stats['actual_p10'] = np.percentile(weighted_prices, 10)
            stats['actual_p90'] = np.percentile(weighted_prices, 90)
            stats['actual_p95'] = np.percentile(weighted_prices, 95)
            
            results.append(stats)
        
        # Convert to DataFrames
        results_df = pd.DataFrame(results)
        results_df = results_df.sort_values(['month_order', 'day']).reset_index(drop=True)
        results_df = results_df.drop('month_order', axis=1)
        
        print(f"   ✓ Calculated statistics for {len(results_df)} daily slots")
        
        return results_df
    
    def calculate_monthly_statistics(self, df_filtered, month_order_map):
        """
        Calculate monthly GENERATION-WEIGHTED price statistics with compression
        """
        print("\n📊 Calculating MONTHLY generation-weighted price statistics...")
        
        # Calculate monthly weighted prices
        def calc_weighted_monthly(group):
            mask = group['generation_mw'] > 0
            if mask.sum() == 0:
                return pd.Series({
                    'weighted_price': np.nan,
                    'total_generation': 0,
                    'generating_hours': 0
                })
            
            gen_positive = group.loc[mask]
            weighted_price = (gen_positive['price'] * gen_positive['generation_mw']).sum() / gen_positive['generation_mw'].sum()
            
            return pd.Series({
                'weighted_price': weighted_price,
                'total_generation': group['generation_mw'].sum(),
                'generating_hours': mask.sum()
            })
        
        # Calculate monthly aggregates for each year
        monthly_data = df_filtered.groupby(['year', 'month']).apply(calc_weighted_monthly).reset_index()
        
        # Calculate statistics across years for each month
        grouped = monthly_data.groupby('month')
        
        results = []
        
        for month, group in grouped:
            valid_group = group[group['weighted_price'].notna()]
            if len(valid_group) < 5:
                continue
            
            weighted_prices = valid_group['weighted_price'].values
            
            month_idx = int(month)
            
            # Determine the forecast year
            forecast_year = self.get_forecast_year(month_idx)
            
            # Apply compression
            compressed_prices, compression_info = self.compress_outliers(
                weighted_prices,
                self.compression_lower,
                self.compression_upper
            )
            
            stats = {
                'month_name': self.month_names_full[month_idx],
                'year': forecast_year,  # ADD YEAR
                'month': month_idx,
                'month_order': month_order_map[month_idx],
                'mean': np.mean(compressed_prices),
                'std_dev': np.std(compressed_prices),
                'count': len(valid_group),
                'avg_generation_mwh': valid_group['total_generation'].mean(),
                'avg_generating_hours': valid_group['generating_hours'].mean()
            }
            
            # Percentiles on compressed data
            for p in self.full_percentiles:
                stats[f'p{p}'] = np.percentile(compressed_prices, p)
            
            # Actual values
            stats['actual_mean'] = np.mean(weighted_prices)
            stats['actual_min'] = np.min(weighted_prices)
            stats['actual_max'] = np.max(weighted_prices)
            stats['actual_p5'] = np.percentile(weighted_prices, 5)
            stats['actual_p10'] = np.percentile(weighted_prices, 10)
            stats['actual_p90'] = np.percentile(weighted_prices, 90)
            stats['actual_p95'] = np.percentile(weighted_prices, 95)
            
            results.append(stats)
        
        # Convert to DataFrames
        results_df = pd.DataFrame(results)
        results_df = results_df.sort_values(['month_order']).reset_index(drop=True)
        results_df = results_df.drop('month_order', axis=1)
        
        print(f"   ✓ Calculated statistics for {len(results_df)} months")
        
        return results_df
    
    def create_hourly_timeseries(self, df_filtered, month_order_map):
        """
        Create hourly price timeseries with years as columns
        """
        print("\n⏰ Creating HOURLY price timeseries...")
        
        # Use all available data - no year skipping
        df_work = df_filtered.copy()
        
        df_work['month_order'] = df_work['month'].map(month_order_map)
        
        # Pivot for prices
        pivot_df = df_work.pivot_table(
            index=['month', 'day', 'hour', 'month_order'],
            columns='year',
            values='price',
            aggfunc='mean'
        ).reset_index()
        
        pivot_df = pivot_df.sort_values(['month_order', 'day', 'hour']).reset_index(drop=True)
        
        pivot_df['datetime_label'] = pivot_df.apply(
            lambda row: f"{self.month_names[int(row['month'])]}-{int(row['day']):02d} {int(row['hour']):02d}:00",
            axis=1
        )
        
        pivot_df = pivot_df.drop('month_order', axis=1)
        
        year_cols = [col for col in pivot_df.columns if isinstance(col, int)]
        cols = ['datetime_label', 'month', 'day', 'hour'] + sorted(year_cols)
        pivot_df = pivot_df[cols]
        
        print(f"   ✓ Created timeseries for {len(pivot_df)} hourly slots")
        print(f"   ✓ Including data from all {len(year_cols)} years")
        
        return pivot_df
    
    def create_hourly_timeseries_compressed(self, df_filtered, month_order_map):
        """
        Create hourly price timeseries with COMPRESSED values
        MODIFIED: Uses P25-P75 compression instead of P10-P90
        """
        print("\n🗜️  Creating COMPRESSED hourly price timeseries...")
        
        # LOCAL VARIABLES - won't affect class-level compression settings
        local_compression_lower = 25  # P25 instead of P10
        local_compression_upper = 75  # P75 instead of P90
        
        # Use all available data - no year skipping
        df_work = df_filtered.copy()
        
        # First, calculate P25 and P75 for each hour slot across all years
        compression_params = {}
        grouped = df_work.groupby(['month', 'day', 'hour'])
        
        for (month, day, hour), group in grouped:
            if len(group) >= 5:  # Need enough data points
                prices = group['price'].values
                # Use LOCAL variables here, not class variables
                P_lower = np.percentile(prices, local_compression_lower)  # P25
                P_upper = np.percentile(prices, local_compression_upper)  # P75
                compression_params[(month, day, hour)] = (P_lower, P_upper)
        
        # Now apply compression to each individual value
        def compress_value(row):
            key = (row['month'], row['day'], row['hour'])
            if key in compression_params:
                P_lower, P_upper = compression_params[key]
                value = row['price']
                
                if value < P_lower:
                    offset = P_lower - value
                    return P_lower - np.log1p(offset)
                elif value > P_upper:
                    offset = value - P_upper
                    return P_upper + np.log1p(offset)
                else:
                    return value
            else:
                return row['price']  # No compression if insufficient data
        
        df_work['compressed_price'] = df_work.apply(compress_value, axis=1)
        df_work['month_order'] = df_work['month'].map(month_order_map)
        
        # Pivot for compressed prices
        pivot_df = df_work.pivot_table(
            index=['month', 'day', 'hour', 'month_order'],
            columns='year',
            values='compressed_price',
            aggfunc='mean'
        ).reset_index()
        
        pivot_df = pivot_df.sort_values(['month_order', 'day', 'hour']).reset_index(drop=True)
        
        pivot_df['datetime_label'] = pivot_df.apply(
            lambda row: f"{self.month_names[int(row['month'])]}-{int(row['day']):02d} {int(row['hour']):02d}:00",
            axis=1
        )
        
        pivot_df = pivot_df.drop('month_order', axis=1)
        
        year_cols = [col for col in pivot_df.columns if isinstance(col, int)]
        cols = ['datetime_label', 'month', 'day', 'hour'] + sorted(year_cols)
        pivot_df = pivot_df[cols]
        
        print(f"   ✓ Created compressed timeseries for {len(pivot_df)} hourly slots")
        print(f"   ✓ Including data from all {len(year_cols)} years")
        print(f"   📌 Using P25-P75 compression (more aggressive than statistics files)")
        
        return pivot_df
    
    def create_daily_timeseries(self, df_filtered, month_order_map):
        """
        Create daily GENERATION-WEIGHTED price timeseries
        """
        print("\n📅 Creating DAILY generation-weighted price timeseries...")
        
        # Calculate weighted daily prices
        def calc_weighted_price(group):
            mask = group['generation_mw'] > 0
            if mask.sum() == 0:
                return np.nan
            gen_positive = group.loc[mask]
            return (gen_positive['price'] * gen_positive['generation_mw']).sum() / gen_positive['generation_mw'].sum()
        
        df_daily = df_filtered.groupby(['year', 'month', 'day']).apply(calc_weighted_price).reset_index()
        df_daily.columns = ['year', 'month', 'day', 'weighted_price']
        
        # Use all available data - no year skipping
        
        df_daily['month_order'] = df_daily['month'].map(month_order_map)
        
        pivot_df = df_daily.pivot_table(
            index=['month', 'day', 'month_order'],
            columns='year',
            values='weighted_price',
            aggfunc='first'
        ).reset_index()
        
        pivot_df = pivot_df.sort_values(['month_order', 'day']).reset_index(drop=True)
        
        pivot_df['date_label'] = pivot_df.apply(
            lambda row: f"{self.month_names[int(row['month'])]}-{int(row['day']):02d}",
            axis=1
        )
        
        pivot_df = pivot_df.drop('month_order', axis=1)
        
        year_cols = [col for col in pivot_df.columns if isinstance(col, int)]
        cols = ['date_label', 'month', 'day'] + sorted(year_cols)
        pivot_df = pivot_df[cols]
        
        print(f"   ✓ Created timeseries for {len(pivot_df)} daily slots")
        print(f"   ✓ Including data from all {len(year_cols)} years")
        
        return pivot_df
    
    def create_monthly_timeseries(self, df_filtered, month_order_map):
        """
        Create monthly GENERATION-WEIGHTED price timeseries
        """
        print("\n📊 Creating MONTHLY generation-weighted price timeseries...")
        
        # Calculate weighted monthly prices
        def calc_weighted_price(group):
            mask = group['generation_mw'] > 0
            if mask.sum() == 0:
                return np.nan
            gen_positive = group.loc[mask]
            return (gen_positive['price'] * gen_positive['generation_mw']).sum() / gen_positive['generation_mw'].sum()
        
        df_monthly = df_filtered.groupby(['year', 'month']).apply(calc_weighted_price).reset_index()
        df_monthly.columns = ['year', 'month', 'weighted_price']
        
        # Use all available data - no year skipping
        
        df_monthly['month_order'] = df_monthly['month'].map(month_order_map)
        
        pivot_df = df_monthly.pivot_table(
            index=['month', 'month_order'],
            columns='year',
            values='weighted_price',
            aggfunc='first'
        ).reset_index()
        
        pivot_df = pivot_df.sort_values(['month_order']).reset_index(drop=True)
        
        pivot_df['month_name'] = pivot_df['month'].apply(lambda x: self.month_names_full[int(x)])
        
        pivot_df = pivot_df.drop('month_order', axis=1)
        
        year_cols = [col for col in pivot_df.columns if isinstance(col, int)]
        cols = ['month_name', 'month'] + sorted(year_cols)
        pivot_df = pivot_df[cols]
        
        print(f"   ✓ Created timeseries for {len(pivot_df)} months")
        print(f"   ✓ Including data from all {len(year_cols)} years")
        
        return pivot_df
    
    def save_all_results(self, hourly_stats, daily_stats, monthly_stats, 
                        hourly_ts, hourly_ts_compressed, daily_ts, monthly_ts, site_name):
        """
        Save all files in the new structure: Renewable Portfolio LLC/{site_name}/Price/
        """
        # Create the site-specific Price folder
        price_path = self.base_output_path / site_name / 'Price'
        price_path.mkdir(parents=True, exist_ok=True)
        
        # Create plots folder for the site (if not already created by generation script)
        plots_path = self.base_output_path / site_name / 'plots'
        plots_path.mkdir(exist_ok=True)
        
        # Save hourly statistics (CHANGED: stats -> forecast)
        hourly_file = f"{site_name}_price_hourly_forecast.csv"
        hourly_path = price_path / hourly_file
        
        cols_hourly = ['datetime_label', 'year', 'month', 'day', 'hour', 'mean', 'std_dev'] + \
                     [f'p{p}' for p in self.full_percentiles] + \
                     ['actual_mean', 'actual_min', 'actual_max', 'actual_p5', 'actual_p10', 
                      'actual_p90', 'actual_p95', 'count', 'avg_generation']
        
        hourly_stats[cols_hourly].to_csv(hourly_path, index=False, float_format='%.3f')
        print(f"\n💾 Saved: Renewable Portfolio LLC/{site_name}/Price/{hourly_file}")
        
        # Save daily statistics (CHANGED: stats -> forecast)
        daily_file = f"{site_name}_price_daily_forecast.csv"
        daily_path = price_path / daily_file
        
        cols_daily = ['date_label', 'year', 'month', 'day', 'mean', 'std_dev'] + \
                    [f'p{p}' for p in self.full_percentiles] + \
                    ['actual_mean', 'actual_min', 'actual_max', 'actual_p5', 'actual_p10',
                     'actual_p90', 'actual_p95', 'count', 'avg_generation_mwh', 'avg_generating_hours']
        
        daily_stats[cols_daily].to_csv(daily_path, index=False, float_format='%.3f')
        print(f"💾 Saved: Renewable Portfolio LLC/{site_name}/Price/{daily_file}")
        
        # Save monthly statistics (CHANGED: stats -> forecast)
        monthly_file = f"{site_name}_price_monthly_forecast.csv"
        monthly_path = price_path / monthly_file
        
        cols_monthly = ['month_name', 'year', 'month', 'mean', 'std_dev'] + \
                      [f'p{p}' for p in self.full_percentiles] + \
                      ['actual_mean', 'actual_min', 'actual_max', 'actual_p5', 'actual_p10',
                       'actual_p90', 'actual_p95', 'count', 'avg_generation_mwh', 'avg_generating_hours']
        
        monthly_stats[cols_monthly].to_csv(monthly_path, index=False, float_format='%.3f')
        print(f"💾 Saved: Renewable Portfolio LLC/{site_name}/Price/{monthly_file}")
        
        # Save hourly timeseries (original values)
        hourly_ts_file = f"{site_name}_price_hourly_timeseries.csv"
        hourly_ts_path = price_path / hourly_ts_file
        hourly_ts_save = hourly_ts.copy()
        year_cols = [col for col in hourly_ts_save.columns if isinstance(col, int)]
        for col in year_cols:
            hourly_ts_save[col] = hourly_ts_save[col].apply(lambda x: '' if pd.isna(x) else f'{x:.2f}')
        hourly_ts_save.to_csv(hourly_ts_path, index=False)
        print(f"💾 Saved: Renewable Portfolio LLC/{site_name}/Price/{hourly_ts_file}")
        
        # Save hourly COMPRESSED timeseries
        hourly_compressed_file = f"{site_name}_price_hourly_timeseries_compressed.csv"
        hourly_compressed_path = price_path / hourly_compressed_file
        hourly_ts_compressed_save = hourly_ts_compressed.copy()
        year_cols = [col for col in hourly_ts_compressed_save.columns if isinstance(col, int)]
        for col in year_cols:
            hourly_ts_compressed_save[col] = hourly_ts_compressed_save[col].apply(lambda x: '' if pd.isna(x) else f'{x:.2f}')
        hourly_ts_compressed_save.to_csv(hourly_compressed_path, index=False)
        print(f"💾 Saved: Renewable Portfolio LLC/{site_name}/Price/{hourly_compressed_file}")
        
        # Save daily timeseries (weighted)
        daily_ts_file = f"{site_name}_price_daily_timeseries.csv"
        daily_ts_path = price_path / daily_ts_file
        daily_ts_save = daily_ts.copy()
        year_cols = [col for col in daily_ts_save.columns if isinstance(col, int)]
        for col in year_cols:
            daily_ts_save[col] = daily_ts_save[col].apply(lambda x: '' if pd.isna(x) else f'{x:.2f}')
        daily_ts_save.to_csv(daily_ts_path, index=False)
        print(f"💾 Saved: Renewable Portfolio LLC/{site_name}/Price/{daily_ts_file}")
        
        # Save monthly timeseries (weighted)
        monthly_ts_file = f"{site_name}_price_monthly_timeseries.csv"
        monthly_ts_path = price_path / monthly_ts_file
        monthly_ts_save = monthly_ts.copy()
        year_cols = [col for col in monthly_ts_save.columns if isinstance(col, int)]
        for col in year_cols:
            monthly_ts_save[col] = monthly_ts_save[col].apply(lambda x: '' if pd.isna(x) else f'{x:.2f}')
        monthly_ts_save.to_csv(monthly_ts_path, index=False)
        print(f"💾 Saved: Renewable Portfolio LLC/{site_name}/Price/{monthly_ts_file}")
    
    def print_sample_results(self, hourly_stats, daily_stats):
        """
        Print formatted sample results showing compression effect
        """
        print("\n" + "="*60)
        print("SAMPLE RESULTS WITH COMPRESSION")
        print("="*60)
        
        # Sample hourly statistics
        print("\n⚡ HOURLY PRICE STATISTICS SAMPLE:")
        if not hourly_stats.empty:
            mid_month = hourly_stats['month'].min()
            sample = hourly_stats[(hourly_stats['month'] == mid_month) & 
                                (hourly_stats['day'] == 15) & 
                                (hourly_stats['hour'] == 12)]
            
            if not sample.empty:
                row = sample.iloc[0]
                
                print(f"\n   {row['datetime_label']} (Year: {row['year']}):")
                print(f"   Original range: ${row['actual_min']:.2f} to ${row['actual_max']:.2f}")
                print(f"   Mean: ${row['mean']:.2f}, P10-P90: ${row['p10']:.2f}-${row['p90']:.2f}")
                print(f"   (Statistics based on compressed values)")
        
        # Sample daily statistics
        print("\n📅 DAILY WEIGHTED PRICE STATISTICS SAMPLE:")
        if not daily_stats.empty:
            mid_month = daily_stats['month'].min()
            sample = daily_stats[(daily_stats['month'] == mid_month) & (daily_stats['day'] == 15)]
            
            if not sample.empty:
                row = sample.iloc[0]
                print(f"\n   {row['date_label']} (Year: {row['year']}):")
                print(f"   Weighted avg: ${row['mean']:.2f}, P10-P90: ${row['p10']:.2f}-${row['p90']:.2f}")
                print(f"   Based on {row['avg_generating_hours']:.1f} generating hours/day")
    
    def process_single_site(self, site_name, start_month, end_month):
        """
        Process a single site with given month range
        """
        # Create month order mapping
        month_order_map = self.create_month_order_map(start_month, end_month)
        
        # Load and prepare data
        print(f"\n{'='*60}")
        print(f"Processing: {site_name}")
        
        months_in_range = self.get_months_in_range(start_month, end_month)
        num_months = len(months_in_range)
        
        if start_month <= end_month:
            print(f"Month range: {self.month_names[start_month]} to {self.month_names[end_month]} ({num_months} months)")
        else:
            print(f"Month range: {self.month_names[start_month]} to {self.month_names[end_month]} (year-wrapping, {num_months} months)")
        
        print(f"{'='*60}")
        
        # Use the mapping to get the actual filename
        actual_filename = self.site_file_map.get(site_name)
        if actual_filename:
            file_path = self.data_path / actual_filename
        else:
            # Fallback - try the standard naming convention
            file_path = self.data_path / f"{site_name}_generation_price_combined.csv"
            
        print(f"\n📁 Loading data from: {file_path.name}")
        
        try:
            df = pd.read_csv(file_path)
            df['datetime'] = pd.to_datetime(df['datetime'])
            
            # Extract day of month from datetime since it's not in the columns
            df['day'] = df['datetime'].dt.day
            
            # Ensure all columns are integers
            df['year'] = df['year'].astype(int)
            df['month'] = df['month'].astype(int)
            df['hour'] = df['hour'].astype(int)
            
            # Filter for selected months
            df_filtered = self.filter_data_for_months(df, start_month, end_month)
            
            # Get data summary
            years_available = sorted(df_filtered['year'].unique())
            print(f"\n📊 Data summary:")
            print(f"   Years available: {years_available[0]} to {years_available[-1]} ({len(years_available)} years)")
            print(f"   Total data points: {len(df_filtered):,}")
            print(f"   Price range: ${df_filtered['price'].min():.2f} to ${df_filtered['price'].max():.2f}")
            print(f"   Negative price hours: {(df_filtered['price'] < 0).sum()} ({(df_filtered['price'] < 0).sum()/len(df_filtered)*100:.1f}%)")
            
            # Updated note about year-wrapping
            if start_month > end_month:
                print(f"\n   ℹ️  Note: Year-wrapping range detected!")
                print(f"   First year ({years_available[0]}) and last year ({years_available[-1]}) may have partial data")
                print(f"   All available data will be included in the timeseries files")
            
            # Calculate statistics at all three levels
            print("\n" + "-"*40)
            print("PRICE DISTRIBUTION STATISTICS")
            print("-"*40)
            
            hourly_stats = self.calculate_hourly_statistics(df_filtered, month_order_map)
            daily_stats = self.calculate_daily_statistics(df_filtered, month_order_map)
            monthly_stats = self.calculate_monthly_statistics(df_filtered, month_order_map)
            
            # Create timeseries
            print("\n" + "-"*40)
            print("PRICE TIMESERIES GENERATION")
            print("-"*40)
            
            hourly_ts = self.create_hourly_timeseries(df_filtered, month_order_map)
            hourly_ts_compressed = self.create_hourly_timeseries_compressed(df_filtered, month_order_map)
            daily_ts = self.create_daily_timeseries(df_filtered, month_order_map)
            monthly_ts = self.create_monthly_timeseries(df_filtered, month_order_map)
            
            # Print samples
            self.print_sample_results(hourly_stats, daily_stats)
            
            # Save all results
            print("\n" + "-"*40)
            print("SAVING RESULTS")
            print("-"*40)
            
            self.save_all_results(hourly_stats, daily_stats, monthly_stats,
                                hourly_ts, hourly_ts_compressed, daily_ts, monthly_ts, site_name)
            
            return True
            
        except Exception as e:
            print(f"\n❌ Error processing {site_name}: {str(e)}")
            return False
    
    def run_simulation(self):
        """
        Main function to run the price simulation
        """
        print("\n💰 Price Simulation with Outlier Compression")
        print("   (Handles extreme prices elegantly)")
        print("="*60)
        
        # Get site selection
        site_selection = self.get_site_selection()
        if not site_selection:
            return
        
        # Get automatic month range
        start_month, end_month = self.get_automatic_month_range()
        
        # Process based on selection
        if site_selection == 'ALL_SITES':
            print("\n" + "="*60)
            print("🚀 PROCESSING ALL SITES")
            print("="*60)
            
            successful = 0
            failed = 0
            
            for i, site_name in enumerate(self.available_sites, 1):
                print(f"\n[{i}/{len(self.available_sites)}] Processing {site_name}...")
                
                if self.process_single_site(site_name, start_month, end_month):
                    successful += 1
                else:
                    failed += 1
            
            # Summary
            print("\n" + "="*60)
            print("✨ ALL SITES PROCESSING COMPLETE!")
            print("="*60)
            print(f"\n📊 Summary:")
            print(f"   ✅ Successfully processed: {successful} sites")
            if failed > 0:
                print(f"   ❌ Failed: {failed} sites")
            
            print(f"\n12-month period: {self.month_names[start_month]} to {self.month_names[end_month]}")
            print(f"\n📁 Files saved in: Renewable Portfolio LLC/[site_name]/Price/")
            
        else:
            # Process single site
            if self.process_single_site(site_selection, start_month, end_month):
                print("\n" + "="*60)
                print("✨ PRICE SIMULATION COMPLETE!")
                print("="*60)
                print(f"\nAnalysis created for {site_selection}")
                
                print(f"12-month period: {self.month_names[start_month]} to {self.month_names[end_month]}")
                
                print(f"\n📁 Files saved in:")
                print(f"   Renewable Portfolio LLC/{site_selection}/Price/")
                print("     • Hourly/Daily/Monthly price statistics (with year column)")
                print("     • Hourly/Daily/Monthly price timeseries")
                print("     • Hourly compressed timeseries for visualization")
                print(f"   Renewable Portfolio LLC/{site_selection}/plots/")
                print("     • (Ready for future visualizations)")
                
                print("\n💡 Key features:")
                print("   • Outliers compressed logarithmically (P10-P90)")
                print("   • Full P1-P99 percentiles available")
                print("   • Actual values tracked for reference")
                print("   • Daily and Monthly use generation-weighted prices")
        
        # Ask if user wants to create another simulation
        another = input("\n🔄 Create another price simulation? (y/n): ").strip().lower()
        if another == 'y':
            self.run_simulation()

# Run the simulation
if __name__ == "__main__":
    simulator = PriceSimulation()
    simulator.run_simulation()


🔧 Initializing Price Simulation...
   Found 7 combined files

💰 Price Simulation with Outlier Compression
   (Handles extreme prices elegantly)

PRICE SIMULATION WITH OUTLIER COMPRESSION

Available options:
0. ALL SITES (Process all sites at once)
1. Blue_Wing_Solar_Energy_Generator
2. High_Lonesome_Wind_Power
3. Midway_Solar_Farm_III
4. Misae_Solar
5. Mount_Signal_Solar_Farm_II
6. RE_Mustang_LLC
7. Stanton_Wind_Energy_LLC

📅 Auto-detected period: Jul to Jun (12 months)
   Starting from current month: Jul

🚀 PROCESSING ALL SITES

[1/7] Processing Blue_Wing_Solar_Energy_Generator...

Processing: Blue_Wing_Solar_Energy_Generator
Month range: Jul to Jun (year-wrapping, 12 months)

📁 Loading data from: Blue_Wing_Solar_Energy_Generator_generation_price_combined.csv

📊 Data summary:
   Years available: 2012 to 2025 (14 years)
   Total data points: 115,115
   Price range: $-165.83 to $988.58
   Negative price hours: 1507 (1.3%)

   ℹ️  Note: Year-wrapping range detected!
   First year (2012)

# Price_da

In [5]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

class PriceDayAheadSimulation:
    """
    Create statistical DAY-AHEAD price profiles with percentile compression for outliers
    Handles extreme prices elegantly while preserving their presence in distributions
    """
    
    def __init__(self):
        print("\n🔧 Initializing Day-Ahead Price Simulation...")
        
        # Define paths - reading from resurety_data folder (UPDATED PATH)
        self.data_path = Path('resurety_data')  # Changed from '../resurety_data'
        self.base_output_path = Path('Renewable Portfolio LLC')  # New base output path
        
        # Get available combined files and create mapping
        self.available_files = list(self.data_path.glob('*_generation_price_combined.csv'))
        self.available_sites = []
        self.site_file_map = {}  # Map clean site names to actual filenames
        
        for f in self.available_files:
            # Store the full filename (without path)
            full_filename = f.name
            
            # Extract site name by removing '_generation_price_combined.csv'
            site_name = f.stem.replace('_generation_price_combined', '')
            
            # Clean up the site name by removing '_hourly' to avoid redundancy in output files
            clean_site_name = site_name.replace('_hourly', '')
            
            self.available_sites.append(clean_site_name)
            # Map the clean name to the actual filename
            self.site_file_map[clean_site_name] = full_filename
        
        print(f"   Found {len(self.available_files)} combined files")
        
        # Define percentiles for FULL distribution (including compressed outliers)
        self.full_percentiles = [1, 5, 10, 25, 50, 75, 90, 95, 99]
        
        # Define compression thresholds (P10 and P90 for compression)
        self.compression_lower = 10
        self.compression_upper = 90
        
        # Define thresholds for extreme event tracking
        self.negative_threshold = 0
        self.spike_threshold = 100
        self.extreme_spike_threshold = 200
        
        # Month names for labeling
        self.month_names = ['', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                           'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
        self.month_names_full = ['', 'January', 'February', 'March', 'April', 'May', 'June',
                                'July', 'August', 'September', 'October', 'November', 'December']
    
    def get_site_selection(self):
        """
        Interactive site selection with option for all sites
        """
        print("\n" + "="*60)
        print("DAY-AHEAD PRICE SIMULATION WITH OUTLIER COMPRESSION")
        print("="*60)
        
        if not self.available_sites:
            print("❌ No combined generation-price files found in resurety_data!")
            return None
        
        print("\nAvailable options:")
        print("0. ALL SITES (Process all sites at once)")
        for i, site in enumerate(self.available_sites):
            print(f"{i+1}. {site}")
        
        print("="*60)
        
        while True:
            try:
                selection = input("\n💰 Select option number (0 for all sites): ").strip()
                if selection == '0':
                    return 'ALL_SITES'
                else:
                    idx = int(selection) - 1
                    if 0 <= idx < len(self.available_sites):
                        return self.available_sites[idx]
                    else:
                        print("❌ Invalid selection!")
            except:
                print("❌ Please enter a valid number!")
    
    def get_automatic_month_range(self):
        """
        Automatically determine month range: current month to 11 months later
        """
        current_date = datetime.now()
        current_month = current_date.month
        
        # Use current month as start
        start_month = current_month
        
        # End month is one month before start month (12 month cycle)
        if start_month == 1:
            end_month = 12
        else:
            end_month = start_month - 1
        
        print(f"\n📅 Auto-detected period: {self.month_names[start_month]} to {self.month_names[end_month]} (12 months)")
        print(f"   Starting from current month: {self.month_names[current_month]}")
        
        return start_month, end_month
    
    def get_months_in_range(self, start_month, end_month):
        """
        Get list of months in range, handling year-wrapping
        """
        if start_month <= end_month:
            return list(range(start_month, end_month + 1))
        else:
            return list(range(start_month, 13)) + list(range(1, end_month + 1))
    
    def filter_data_for_months(self, df, start_month, end_month):
        """
        Filter dataframe for month range, handling year-wrapping
        """
        months_in_range = self.get_months_in_range(start_month, end_month)
        return df[df['month'].isin(months_in_range)].copy()
    
    def create_month_order_map(self, start_month, end_month):
        """
        Create a mapping for sorting months in the specified order
        """
        months_in_range = self.get_months_in_range(start_month, end_month)
        return {month: idx for idx, month in enumerate(months_in_range)}
    
    def get_forecast_year(self, month):
        """
        Determine the forecast year for a given month based on current date
        """
        current_date = datetime.now()
        current_year = current_date.year
        current_month = current_date.month
        
        # If the forecast month is >= current month, it's this year
        # Otherwise it's next year (handling year wrap)
        if month >= current_month:
            return current_year
        else:
            return current_year + 1
    
    def compress_outliers(self, values, lower_pct=10, upper_pct=90):
        """
        Compress outliers using logarithmic compression
        This keeps outliers in the distribution but makes them manageable for visualization
        """
        # Calculate compression thresholds
        P_lower = np.percentile(values, lower_pct)
        P_upper = np.percentile(values, upper_pct)
        
        # Create compressed values array
        compressed = np.copy(values)
        
        # Compress lower tail (negative extremes)
        lower_mask = values < P_lower
        if np.any(lower_mask):
            # Use logarithmic compression for negative tail
            # Shift to make positive for log, then shift back
            offset = P_lower - values[lower_mask]
            compressed[lower_mask] = P_lower - np.log1p(offset)
        
        # Compress upper tail (positive extremes)
        upper_mask = values > P_upper
        if np.any(upper_mask):
            # Use logarithmic compression for positive tail
            offset = values[upper_mask] - P_upper
            compressed[upper_mask] = P_upper + np.log1p(offset)
        
        # Return compressed values and compression info
        compression_info = {
            'P_lower': P_lower,
            'P_upper': P_upper,
            'n_compressed_lower': np.sum(lower_mask),
            'n_compressed_upper': np.sum(upper_mask),
            'original_min': np.min(values),
            'original_max': np.max(values),
            'compressed_min': np.min(compressed),
            'compressed_max': np.max(compressed)
        }
        
        return compressed, compression_info
    
    def calculate_hourly_statistics(self, df_filtered, month_order_map):
        """
        Calculate hourly DAY-AHEAD price statistics with percentile compression
        """
        print("\n⚡ Calculating HOURLY day-ahead price statistics...")
        
        grouped = df_filtered.groupby(['month', 'day', 'hour'])
        
        results = []
        
        for (month, day, hour), group in grouped:
            if len(group) < 5:  # Skip if too few data points
                continue
            
            prices = group['price_da'].values  # Using price_da instead of price
            generation = group['generation_mw'].values
            
            # Determine the forecast year
            forecast_year = self.get_forecast_year(month)
            
            # Apply percentile compression
            compressed_prices, compression_info = self.compress_outliers(
                prices, 
                self.compression_lower, 
                self.compression_upper
            )
            
            # Create datetime label
            datetime_label = f"{self.month_names[month]}-{day:02d} {hour:02d}:00"
            
            # Main statistics on COMPRESSED data for visualization
            stats = {
                'datetime_label': datetime_label,
                'year': forecast_year,  # ADD YEAR
                'month': month,
                'day': day,
                'hour': hour,
                'month_order': month_order_map[month],
                'mean': np.mean(compressed_prices),
                'std_dev': np.std(compressed_prices),
                'count': len(group),
                'avg_generation': np.mean(generation)
            }
            
            # Calculate percentiles on COMPRESSED data
            for p in self.full_percentiles:
                stats[f'p{p}'] = np.percentile(compressed_prices, p)
            
            # Also store ACTUAL percentiles for reference
            stats['actual_mean'] = np.mean(prices)
            stats['actual_min'] = np.min(prices)
            stats['actual_max'] = np.max(prices)
            stats['actual_p5'] = np.percentile(prices, 5)
            stats['actual_p10'] = np.percentile(prices, 10)
            stats['actual_p90'] = np.percentile(prices, 90)
            stats['actual_p95'] = np.percentile(prices, 95)
            
            results.append(stats)
        
        # Convert to DataFrames and sort
        results_df = pd.DataFrame(results)
        results_df = results_df.sort_values(['month_order', 'day', 'hour']).reset_index(drop=True)
        results_df = results_df.drop('month_order', axis=1)
        
        print(f"   ✓ Calculated statistics for {len(results_df)} hourly slots")
        
        return results_df
    
    def calculate_daily_statistics(self, df_filtered, month_order_map):
        """
        Calculate daily GENERATION-WEIGHTED day-ahead price statistics with compression
        """
        print("\n📅 Calculating DAILY generation-weighted day-ahead price statistics...")
        
        # First calculate generation-weighted prices for each day
        def calc_weighted_daily(group):
            mask = group['generation_mw'] > 0
            if mask.sum() == 0:
                return pd.Series({
                    'weighted_price_da': np.nan,
                    'total_generation': 0,
                    'generating_hours': 0
                })
            
            gen_positive = group.loc[mask]
            weighted_price = (gen_positive['price_da'] * gen_positive['generation_mw']).sum() / gen_positive['generation_mw'].sum()
            
            return pd.Series({
                'weighted_price_da': weighted_price,
                'total_generation': group['generation_mw'].sum(),
                'generating_hours': mask.sum()
            })
        
        # Calculate daily aggregates for each year
        daily_data = df_filtered.groupby(['year', 'month', 'day']).apply(calc_weighted_daily).reset_index()
        
        # Now calculate statistics across years for each day
        grouped = daily_data.groupby(['month', 'day'])
        
        results = []
        
        for (month, day), group in grouped:
            valid_group = group[group['weighted_price_da'].notna()]
            if len(valid_group) < 5:
                continue
            
            weighted_prices = valid_group['weighted_price_da'].values
            
            # Determine the forecast year
            forecast_year = self.get_forecast_year(month)
            
            # Apply compression
            compressed_prices, compression_info = self.compress_outliers(
                weighted_prices,
                self.compression_lower,
                self.compression_upper
            )
            
            # Create date label
            date_label = f"{self.month_names[month]}-{day:02d}"
            
            # Main statistics on compressed data
            stats = {
                'date_label': date_label,
                'year': forecast_year,  # ADD YEAR
                'month': month,
                'day': day,
                'month_order': month_order_map[month],
                'mean': np.mean(compressed_prices),
                'std_dev': np.std(compressed_prices),
                'count': len(valid_group),
                'avg_generation_mwh': valid_group['total_generation'].mean(),
                'avg_generating_hours': valid_group['generating_hours'].mean()
            }
            
            # Percentiles on compressed data
            for p in self.full_percentiles:
                stats[f'p{p}'] = np.percentile(compressed_prices, p)
            
            # Actual percentiles for reference
            stats['actual_mean'] = np.mean(weighted_prices)
            stats['actual_min'] = np.min(weighted_prices)
            stats['actual_max'] = np.max(weighted_prices)
            stats['actual_p5'] = np.percentile(weighted_prices, 5)
            stats['actual_p10'] = np.percentile(weighted_prices, 10)
            stats['actual_p90'] = np.percentile(weighted_prices, 90)
            stats['actual_p95'] = np.percentile(weighted_prices, 95)
            
            results.append(stats)
        
        # Convert to DataFrames
        results_df = pd.DataFrame(results)
        results_df = results_df.sort_values(['month_order', 'day']).reset_index(drop=True)
        results_df = results_df.drop('month_order', axis=1)
        
        print(f"   ✓ Calculated statistics for {len(results_df)} daily slots")
        
        return results_df
    
    def calculate_monthly_statistics(self, df_filtered, month_order_map):
        """
        Calculate monthly GENERATION-WEIGHTED day-ahead price statistics with compression
        """
        print("\n📊 Calculating MONTHLY generation-weighted day-ahead price statistics...")
        
        # Calculate monthly weighted prices
        def calc_weighted_monthly(group):
            mask = group['generation_mw'] > 0
            if mask.sum() == 0:
                return pd.Series({
                    'weighted_price_da': np.nan,
                    'total_generation': 0,
                    'generating_hours': 0
                })
            
            gen_positive = group.loc[mask]
            weighted_price = (gen_positive['price_da'] * gen_positive['generation_mw']).sum() / gen_positive['generation_mw'].sum()
            
            return pd.Series({
                'weighted_price_da': weighted_price,
                'total_generation': group['generation_mw'].sum(),
                'generating_hours': mask.sum()
            })
        
        # Calculate monthly aggregates for each year
        monthly_data = df_filtered.groupby(['year', 'month']).apply(calc_weighted_monthly).reset_index()
        
        # Calculate statistics across years for each month
        grouped = monthly_data.groupby('month')
        
        results = []
        
        for month, group in grouped:
            valid_group = group[group['weighted_price_da'].notna()]
            if len(valid_group) < 5:
                continue
            
            weighted_prices = valid_group['weighted_price_da'].values
            
            month_idx = int(month)
            
            # Determine the forecast year
            forecast_year = self.get_forecast_year(month_idx)
            
            # Apply compression
            compressed_prices, compression_info = self.compress_outliers(
                weighted_prices,
                self.compression_lower,
                self.compression_upper
            )
            
            stats = {
                'month_name': self.month_names_full[month_idx],
                'year': forecast_year,  # ADD YEAR
                'month': month_idx,
                'month_order': month_order_map[month_idx],
                'mean': np.mean(compressed_prices),
                'std_dev': np.std(compressed_prices),
                'count': len(valid_group),
                'avg_generation_mwh': valid_group['total_generation'].mean(),
                'avg_generating_hours': valid_group['generating_hours'].mean()
            }
            
            # Percentiles on compressed data
            for p in self.full_percentiles:
                stats[f'p{p}'] = np.percentile(compressed_prices, p)
            
            # Actual values
            stats['actual_mean'] = np.mean(weighted_prices)
            stats['actual_min'] = np.min(weighted_prices)
            stats['actual_max'] = np.max(weighted_prices)
            stats['actual_p5'] = np.percentile(weighted_prices, 5)
            stats['actual_p10'] = np.percentile(weighted_prices, 10)
            stats['actual_p90'] = np.percentile(weighted_prices, 90)
            stats['actual_p95'] = np.percentile(weighted_prices, 95)
            
            results.append(stats)
        
        # Convert to DataFrames
        results_df = pd.DataFrame(results)
        results_df = results_df.sort_values(['month_order']).reset_index(drop=True)
        results_df = results_df.drop('month_order', axis=1)
        
        print(f"   ✓ Calculated statistics for {len(results_df)} months")
        
        return results_df
    
    def create_hourly_timeseries(self, df_filtered, month_order_map):
        """
        Create hourly day-ahead price timeseries with years as columns
        """
        print("\n⏰ Creating HOURLY day-ahead price timeseries...")
        
        # Use all available data - no year skipping
        df_work = df_filtered.copy()
        
        df_work['month_order'] = df_work['month'].map(month_order_map)
        
        # Pivot for day-ahead prices
        pivot_df = df_work.pivot_table(
            index=['month', 'day', 'hour', 'month_order'],
            columns='year',
            values='price_da',  # Using price_da instead of price
            aggfunc='mean'
        ).reset_index()
        
        pivot_df = pivot_df.sort_values(['month_order', 'day', 'hour']).reset_index(drop=True)
        
        pivot_df['datetime_label'] = pivot_df.apply(
            lambda row: f"{self.month_names[int(row['month'])]}-{int(row['day']):02d} {int(row['hour']):02d}:00",
            axis=1
        )
        
        pivot_df = pivot_df.drop('month_order', axis=1)
        
        year_cols = [col for col in pivot_df.columns if isinstance(col, int)]
        cols = ['datetime_label', 'month', 'day', 'hour'] + sorted(year_cols)
        pivot_df = pivot_df[cols]
        
        print(f"   ✓ Created timeseries for {len(pivot_df)} hourly slots")
        print(f"   ✓ Including data from all {len(year_cols)} years")
        
        return pivot_df
    
    def create_daily_timeseries(self, df_filtered, month_order_map):
        """
        Create daily GENERATION-WEIGHTED day-ahead price timeseries
        """
        print("\n📅 Creating DAILY generation-weighted day-ahead price timeseries...")
        
        # Calculate weighted daily prices
        def calc_weighted_price(group):
            mask = group['generation_mw'] > 0
            if mask.sum() == 0:
                return np.nan
            gen_positive = group.loc[mask]
            return (gen_positive['price_da'] * gen_positive['generation_mw']).sum() / gen_positive['generation_mw'].sum()
        
        df_daily = df_filtered.groupby(['year', 'month', 'day']).apply(calc_weighted_price).reset_index()
        df_daily.columns = ['year', 'month', 'day', 'weighted_price_da']
        
        # Use all available data - no year skipping
        
        df_daily['month_order'] = df_daily['month'].map(month_order_map)
        
        pivot_df = df_daily.pivot_table(
            index=['month', 'day', 'month_order'],
            columns='year',
            values='weighted_price_da',
            aggfunc='first'
        ).reset_index()
        
        pivot_df = pivot_df.sort_values(['month_order', 'day']).reset_index(drop=True)
        
        pivot_df['date_label'] = pivot_df.apply(
            lambda row: f"{self.month_names[int(row['month'])]}-{int(row['day']):02d}",
            axis=1
        )
        
        pivot_df = pivot_df.drop('month_order', axis=1)
        
        year_cols = [col for col in pivot_df.columns if isinstance(col, int)]
        cols = ['date_label', 'month', 'day'] + sorted(year_cols)
        pivot_df = pivot_df[cols]
        
        print(f"   ✓ Created timeseries for {len(pivot_df)} daily slots")
        print(f"   ✓ Including data from all {len(year_cols)} years")
        
        return pivot_df
    
    def create_monthly_timeseries(self, df_filtered, month_order_map):
        """
        Create monthly GENERATION-WEIGHTED day-ahead price timeseries
        """
        print("\n📊 Creating MONTHLY generation-weighted day-ahead price timeseries...")
        
        # Calculate weighted monthly prices
        def calc_weighted_price(group):
            mask = group['generation_mw'] > 0
            if mask.sum() == 0:
                return np.nan
            gen_positive = group.loc[mask]
            return (gen_positive['price_da'] * gen_positive['generation_mw']).sum() / gen_positive['generation_mw'].sum()
        
        df_monthly = df_filtered.groupby(['year', 'month']).apply(calc_weighted_price).reset_index()
        df_monthly.columns = ['year', 'month', 'weighted_price_da']
        
        # Use all available data - no year skipping
        
        df_monthly['month_order'] = df_monthly['month'].map(month_order_map)
        
        pivot_df = df_monthly.pivot_table(
            index=['month', 'month_order'],
            columns='year',
            values='weighted_price_da',
            aggfunc='first'
        ).reset_index()
        
        pivot_df = pivot_df.sort_values(['month_order']).reset_index(drop=True)
        
        pivot_df['month_name'] = pivot_df['month'].apply(lambda x: self.month_names_full[int(x)])
        
        pivot_df = pivot_df.drop('month_order', axis=1)
        
        year_cols = [col for col in pivot_df.columns if isinstance(col, int)]
        cols = ['month_name', 'month'] + sorted(year_cols)
        pivot_df = pivot_df[cols]
        
        print(f"   ✓ Created timeseries for {len(pivot_df)} months")
        print(f"   ✓ Including data from all {len(year_cols)} years")
        
        return pivot_df
    
    def save_all_results(self, hourly_stats, daily_stats, monthly_stats, 
                        hourly_ts, daily_ts, monthly_ts, site_name):
        """
        Save all files in the new structure: Renewable Portfolio LLC/{site_name}/Price_da/
        """
        # Create the site-specific Price_da folder
        price_da_path = self.base_output_path / site_name / 'Price_da'
        price_da_path.mkdir(parents=True, exist_ok=True)
        
        # Create plots folder for the site (if not already created by other scripts)
        plots_path = self.base_output_path / site_name / 'plots'
        plots_path.mkdir(exist_ok=True)
        
        # Save hourly statistics (CHANGED: stats -> forecast)
        hourly_file = f"{site_name}_price_da_hourly_forecast.csv"
        hourly_path = price_da_path / hourly_file
        
        cols_hourly = ['datetime_label', 'year', 'month', 'day', 'hour', 'mean', 'std_dev'] + \
                     [f'p{p}' for p in self.full_percentiles] + \
                     ['actual_mean', 'actual_min', 'actual_max', 'actual_p5', 'actual_p10', 
                      'actual_p90', 'actual_p95', 'count', 'avg_generation']
        
        hourly_stats[cols_hourly].to_csv(hourly_path, index=False, float_format='%.3f')
        print(f"\n💾 Saved: Renewable Portfolio LLC/{site_name}/Price_da/{hourly_file}")
        
        # Save daily statistics (CHANGED: stats -> forecast)
        daily_file = f"{site_name}_price_da_daily_forecast.csv"
        daily_path = price_da_path / daily_file
        
        cols_daily = ['date_label', 'year', 'month', 'day', 'mean', 'std_dev'] + \
                    [f'p{p}' for p in self.full_percentiles] + \
                    ['actual_mean', 'actual_min', 'actual_max', 'actual_p5', 'actual_p10',
                     'actual_p90', 'actual_p95', 'count', 'avg_generation_mwh', 'avg_generating_hours']
        
        daily_stats[cols_daily].to_csv(daily_path, index=False, float_format='%.3f')
        print(f"💾 Saved: Renewable Portfolio LLC/{site_name}/Price_da/{daily_file}")
        
        # Save monthly statistics (CHANGED: stats -> forecast)
        monthly_file = f"{site_name}_price_da_monthly_forecast.csv"
        monthly_path = price_da_path / monthly_file
        
        cols_monthly = ['month_name', 'year', 'month', 'mean', 'std_dev'] + \
                      [f'p{p}' for p in self.full_percentiles] + \
                      ['actual_mean', 'actual_min', 'actual_max', 'actual_p5', 'actual_p10',
                       'actual_p90', 'actual_p95', 'count', 'avg_generation_mwh', 'avg_generating_hours']
        
        monthly_stats[cols_monthly].to_csv(monthly_path, index=False, float_format='%.3f')
        print(f"💾 Saved: Renewable Portfolio LLC/{site_name}/Price_da/{monthly_file}")
        
        # Save hourly timeseries
        hourly_ts_file = f"{site_name}_price_da_hourly_timeseries.csv"
        hourly_ts_path = price_da_path / hourly_ts_file
        hourly_ts_save = hourly_ts.copy()
        year_cols = [col for col in hourly_ts_save.columns if isinstance(col, int)]
        for col in year_cols:
            hourly_ts_save[col] = hourly_ts_save[col].apply(lambda x: '' if pd.isna(x) else f'{x:.2f}')
        hourly_ts_save.to_csv(hourly_ts_path, index=False)
        print(f"💾 Saved: Renewable Portfolio LLC/{site_name}/Price_da/{hourly_ts_file}")
        
        # Save daily timeseries
        daily_ts_file = f"{site_name}_price_da_daily_timeseries.csv"
        daily_ts_path = price_da_path / daily_ts_file
        daily_ts_save = daily_ts.copy()
        year_cols = [col for col in daily_ts_save.columns if isinstance(col, int)]
        for col in year_cols:
            daily_ts_save[col] = daily_ts_save[col].apply(lambda x: '' if pd.isna(x) else f'{x:.2f}')
        daily_ts_save.to_csv(daily_ts_path, index=False)
        print(f"💾 Saved: Renewable Portfolio LLC/{site_name}/Price_da/{daily_ts_file}")
        
        # Save monthly timeseries
        monthly_ts_file = f"{site_name}_price_da_monthly_timeseries.csv"
        monthly_ts_path = price_da_path / monthly_ts_file
        monthly_ts_save = monthly_ts.copy()
        year_cols = [col for col in monthly_ts_save.columns if isinstance(col, int)]
        for col in year_cols:
            monthly_ts_save[col] = monthly_ts_save[col].apply(lambda x: '' if pd.isna(x) else f'{x:.2f}')
        monthly_ts_save.to_csv(monthly_ts_path, index=False)
        print(f"💾 Saved: Renewable Portfolio LLC/{site_name}/Price_da/{monthly_ts_file}")
    
    def print_sample_results(self, hourly_stats, daily_stats):
        """
        Print formatted sample results
        """
        print("\n" + "="*60)
        print("SAMPLE DAY-AHEAD RESULTS WITH COMPRESSION")
        print("="*60)
        
        # Sample hourly statistics
        print("\n⚡ HOURLY DAY-AHEAD PRICE STATISTICS SAMPLE:")
        if not hourly_stats.empty:
            mid_month = hourly_stats['month'].min()
            sample = hourly_stats[(hourly_stats['month'] == mid_month) & 
                                (hourly_stats['day'] == 15) & 
                                (hourly_stats['hour'] == 12)]
            
            if not sample.empty:
                row = sample.iloc[0]
                
                print(f"\n   {row['datetime_label']} (Year: {row['year']}):")
                print(f"   Original range: ${row['actual_min']:.2f} to ${row['actual_max']:.2f}")
                print(f"   Mean: ${row['mean']:.2f}, P10-P90: ${row['p10']:.2f}-${row['p90']:.2f}")
                print(f"   (Statistics based on compressed values)")
        
        # Sample daily statistics
        print("\n📅 DAILY WEIGHTED DAY-AHEAD PRICE STATISTICS SAMPLE:")
        if not daily_stats.empty:
            mid_month = daily_stats['month'].min()
            sample = daily_stats[(daily_stats['month'] == mid_month) & (daily_stats['day'] == 15)]
            
            if not sample.empty:
                row = sample.iloc[0]
                print(f"\n   {row['date_label']} (Year: {row['year']}):")
                print(f"   Weighted avg: ${row['mean']:.2f}, P10-P90: ${row['p10']:.2f}-${row['p90']:.2f}")
                print(f"   Based on {row['avg_generating_hours']:.1f} generating hours/day")
    
    def process_single_site(self, site_name, start_month, end_month):
        """
        Process a single site with given month range
        """
        # Create month order mapping
        month_order_map = self.create_month_order_map(start_month, end_month)
        
        # Load and prepare data
        print(f"\n{'='*60}")
        print(f"Processing: {site_name}")
        
        months_in_range = self.get_months_in_range(start_month, end_month)
        num_months = len(months_in_range)
        
        if start_month <= end_month:
            print(f"Month range: {self.month_names[start_month]} to {self.month_names[end_month]} ({num_months} months)")
        else:
            print(f"Month range: {self.month_names[start_month]} to {self.month_names[end_month]} (year-wrapping, {num_months} months)")
        
        print(f"{'='*60}")
        
        # Use the mapping to get the actual filename
        actual_filename = self.site_file_map.get(site_name)
        if actual_filename:
            file_path = self.data_path / actual_filename
        else:
            # Fallback - try the standard naming convention
            file_path = self.data_path / f"{site_name}_generation_price_combined.csv"
            
        print(f"\n📁 Loading data from: {file_path.name}")
        
        try:
            df = pd.read_csv(file_path)
            df['datetime'] = pd.to_datetime(df['datetime'])
            
            # Extract day of month from datetime since it's not in the columns
            df['day'] = df['datetime'].dt.day
            
            # Ensure all columns are integers
            df['year'] = df['year'].astype(int)
            df['month'] = df['month'].astype(int)
            df['hour'] = df['hour'].astype(int)
            
            # Filter for selected months
            df_filtered = self.filter_data_for_months(df, start_month, end_month)
            
            # Get data summary
            years_available = sorted(df_filtered['year'].unique())
            print(f"\n📊 Data summary:")
            print(f"   Years available: {years_available[0]} to {years_available[-1]} ({len(years_available)} years)")
            print(f"   Total data points: {len(df_filtered):,}")
            print(f"   Day-ahead price range: ${df_filtered['price_da'].min():.2f} to ${df_filtered['price_da'].max():.2f}")
            print(f"   Negative day-ahead price hours: {(df_filtered['price_da'] < 0).sum()} ({(df_filtered['price_da'] < 0).sum()/len(df_filtered)*100:.1f}%)")
            
            # Updated note about year-wrapping
            if start_month > end_month:
                print(f"\n   ℹ️  Note: Year-wrapping range detected!")
                print(f"   First year ({years_available[0]}) and last year ({years_available[-1]}) may have partial data")
                print(f"   All available data will be included in the timeseries files")
            
            # Calculate statistics at all three levels
            print("\n" + "-"*40)
            print("DAY-AHEAD PRICE DISTRIBUTION STATISTICS")
            print("-"*40)
            
            hourly_stats = self.calculate_hourly_statistics(df_filtered, month_order_map)
            daily_stats = self.calculate_daily_statistics(df_filtered, month_order_map)
            monthly_stats = self.calculate_monthly_statistics(df_filtered, month_order_map)
            
            # Create timeseries
            print("\n" + "-"*40)
            print("DAY-AHEAD PRICE TIMESERIES GENERATION")
            print("-"*40)
            
            hourly_ts = self.create_hourly_timeseries(df_filtered, month_order_map)
            daily_ts = self.create_daily_timeseries(df_filtered, month_order_map)
            monthly_ts = self.create_monthly_timeseries(df_filtered, month_order_map)
            
            # Print samples
            self.print_sample_results(hourly_stats, daily_stats)
            
            # Save all results
            print("\n" + "-"*40)
            print("SAVING RESULTS")
            print("-"*40)
            
            self.save_all_results(hourly_stats, daily_stats, monthly_stats,
                                hourly_ts, daily_ts, monthly_ts, site_name)
            
            return True
            
        except Exception as e:
            print(f"\n❌ Error processing {site_name}: {str(e)}")
            return False
    
    def run_simulation(self):
        """
        Main function to run the day-ahead price simulation
        """
        print("\n💰 Day-Ahead Price Simulation with Outlier Compression")
        print("   (Handles extreme day-ahead prices elegantly)")
        print("="*60)
        
        # Get site selection
        site_selection = self.get_site_selection()
        if not site_selection:
            return
        
        # Get automatic month range
        start_month, end_month = self.get_automatic_month_range()
        
        # Process based on selection
        if site_selection == 'ALL_SITES':
            print("\n" + "="*60)
            print("🚀 PROCESSING ALL SITES")
            print("="*60)
            
            successful = 0
            failed = 0
            
            for i, site_name in enumerate(self.available_sites, 1):
                print(f"\n[{i}/{len(self.available_sites)}] Processing {site_name}...")
                
                if self.process_single_site(site_name, start_month, end_month):
                    successful += 1
                else:
                    failed += 1
            
            # Summary
            print("\n" + "="*60)
            print("✨ ALL SITES PROCESSING COMPLETE!")
            print("="*60)
            print(f"\n📊 Summary:")
            print(f"   ✅ Successfully processed: {successful} sites")
            if failed > 0:
                print(f"   ❌ Failed: {failed} sites")
            
            print(f"\n12-month period: {self.month_names[start_month]} to {self.month_names[end_month]}")
            print(f"\n📁 Files saved in: Renewable Portfolio LLC/[site_name]/Price_da/")
            
        else:
            # Process single site
            if self.process_single_site(site_selection, start_month, end_month):
                print("\n" + "="*60)
                print("✨ DAY-AHEAD PRICE SIMULATION COMPLETE!")
                print("="*60)
                print(f"\nAnalysis created for {site_selection}")
                
                print(f"12-month period: {self.month_names[start_month]} to {self.month_names[end_month]}")
                
                print(f"\n📁 Files saved in:")
                print(f"   Renewable Portfolio LLC/{site_selection}/Price_da/")
                print("     • Hourly/Daily/Monthly day-ahead price statistics (with year column)")
                print("     • Hourly/Daily/Monthly day-ahead price timeseries")
                print(f"   Renewable Portfolio LLC/{site_selection}/plots/")
                print("     • (Ready for future visualizations)")
                
                print("\n💡 Key features:")
                print("   • Outliers compressed logarithmically (P10-P90)")
                print("   • Full P1-P99 percentiles available")
                print("   • Actual values tracked for reference")
                print("   • Daily and Monthly use generation-weighted prices")
        
        # Ask if user wants to create another simulation
        another = input("\n🔄 Create another day-ahead price simulation? (y/n): ").strip().lower()
        if another == 'y':
            self.run_simulation()

# Run the simulation
if __name__ == "__main__":
    simulator = PriceDayAheadSimulation()
    simulator.run_simulation()


🔧 Initializing Day-Ahead Price Simulation...
   Found 7 combined files

💰 Day-Ahead Price Simulation with Outlier Compression
   (Handles extreme day-ahead prices elegantly)

DAY-AHEAD PRICE SIMULATION WITH OUTLIER COMPRESSION

Available options:
0. ALL SITES (Process all sites at once)
1. Blue_Wing_Solar_Energy_Generator
2. High_Lonesome_Wind_Power
3. Midway_Solar_Farm_III
4. Misae_Solar
5. Mount_Signal_Solar_Farm_II
6. RE_Mustang_LLC
7. Stanton_Wind_Energy_LLC

📅 Auto-detected period: Jul to Jun (12 months)
   Starting from current month: Jul

🚀 PROCESSING ALL SITES

[1/7] Processing Blue_Wing_Solar_Energy_Generator...

Processing: Blue_Wing_Solar_Energy_Generator
Month range: Jul to Jun (year-wrapping, 12 months)

📁 Loading data from: Blue_Wing_Solar_Energy_Generator_generation_price_combined.csv

📊 Data summary:
   Years available: 2012 to 2025 (14 years)
   Total data points: 115,115
   Day-ahead price range: $-27.74 to $999.04
   Negative day-ahead price hours: 80 (0.1%)

   ℹ️ 